In [1]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
#Set Column View
pd.set_option('display.max_columns',1000)

In [2]:
draftees = []
draftposit = []
draftyear = []
for year in range(2000,2019):
    website_url = requests.get('https://www.basketball-reference.com/draft/NBA_'+str(year)+'.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_rookie = soup.findAll('td',{'class':"left"})
    count = 1
    listing = int((len(My_rookie)/3))
    for i in range(0,listing):
        draftposit.append(i+1)
        draftees.append(My_rookie[count].get_text())
        draftyear.append(year)
        count = count + 3
print(draftees)

['Kenyon Martin', 'Stromile Swift', 'Darius Miles', 'Marcus Fizer', 'Mike Miller', 'DerMarr Johnson', 'Chris Mihm', 'Jamal Crawford', 'Joel Przybilla', 'Keyon Dooling', 'Jérôme Moïso', 'Etan Thomas', 'Courtney Alexander', 'Mateen Cleaves', 'Jason Collier', 'Hedo Türkoğlu', 'Desmond Mason', 'Quentin Richardson', 'Jamaal Magloire', 'Speedy Claxton', 'Morris Peterson', 'Donnell Harvey', 'DeShawn Stevenson', 'Dalibor Bagarić', 'Jake Tsakalidis', "Mamadou N'Diaye", 'Primož Brezec', 'Erick Barkley', 'Mark Madsen', 'Marko Jarić', 'Dan Langhi', 'A.J. Guyton', 'Jake Voskuhl', 'Khalid El-Amin', 'Mike Smith', 'Soumaila Samake', 'Eddie House', 'Eduardo Nájera', 'Lavor Postell', 'Hanno Möttölä', 'Chris Carrawell', 'Olumide Oyedeji', 'Michael Redd', 'Brian Cardinal', 'Jabari Smith', 'DeeAndre Hulett', 'Josip Sesar', 'Mark Karcher', 'Jason Hart', 'Kaniel Dickens', 'Igor Rakočević', 'Ernest Brown', 'Dan McClintock', 'Cory Hightower', 'Chris Porter', 'Jaquay Walls', 'Scoonie Penn', 'Pete Mickeal', 'Kwa

In [3]:
len(draftyear),len(draftees)

(1129, 1129)

In [6]:
data = []
count = 1
for year in range(2000,2020):
    website_url = requests.get('https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_per_game.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':"per_game_stats"})
    tabledata = My_table.findAll('td')
    for cell in tabledata:
        count = count+1
        if (count%29 == 2):
            data.append(str(year))
        data.append(cell.get_text())

In [7]:
data2 = np.array(data)
refined = np.reshape(data2, (-1, 30))

In [8]:
tableheader = My_table.findAll('th')
headers = []
count = 0
for item in tableheader:
    if (count < 30):
        headers.append(item.get_text())
        count=count+1
headers.remove('Rk')
headers = ['Year'] + headers
print(headers)

['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


In [9]:
df1 = pd.DataFrame(refined, columns=headers)
df1['Player'] = df1['Player'].str.replace('*','')
df1

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2000,Tariq Abdul-Wahad,SG,25,TOT,61,56,25.9,4.5,10.6,.424,0.0,0.4,.130,4.4,10.2,.435,.426,2.4,3.2,.756,1.7,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4
1,2000,Tariq Abdul-Wahad,SG,25,ORL,46,46,26.2,4.8,11.2,.433,0.0,0.5,.095,4.8,10.7,.447,.435,2.5,3.3,.762,1.7,3.5,5.2,1.6,1.2,0.3,1.9,2.5,12.2
2,2000,Tariq Abdul-Wahad,SG,25,DEN,15,10,24.9,3.4,8.7,.389,0.1,0.1,.500,3.3,8.6,.388,.393,2.1,2.8,.738,1.6,1.9,3.5,1.7,0.4,0.8,1.3,2.1,8.9
3,2000,Shareef Abdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,.465,0.4,1.2,.302,6.9,14.4,.478,.477,5.4,6.7,.809,2.7,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3
4,2000,Cory Alexander,PG,26,DEN,29,2,11.3,1.0,3.4,.286,0.3,1.2,.257,0.7,2.2,.302,.332,0.6,0.8,.773,0.3,1.2,1.4,2.0,0.8,0.1,1.0,1.3,2.8
5,2000,Ray Allen,SG,24,MIL,82,82,37.4,7.8,17.2,.455,2.1,5.0,.423,5.7,12.2,.468,.516,4.3,4.9,.887,1.0,3.4,4.4,3.8,1.3,0.2,2.2,2.3,22.1
6,2000,Rafer Alston,PG,23,MIL,27,0,13.4,1.0,3.5,.284,0.1,0.5,.214,0.9,3.0,.296,.300,0.1,0.1,.750,0.2,0.7,0.9,2.6,0.4,0.0,1.1,1.1,2.2
7,2000,John Amaechi,C,29,ORL,80,53,21.1,3.8,8.8,.437,0.0,0.1,.167,3.8,8.7,.439,.438,2.8,3.6,.766,0.8,2.6,3.3,1.2,0.4,0.5,1.7,2.0,10.5
8,2000,Derek Anderson,SG,25,LAC,64,58,34.4,5.9,13.4,.438,0.9,2.8,.309,5.0,10.7,.472,.470,4.2,4.8,.877,1.3,2.8,4.0,3.4,1.4,0.2,2.6,2.3,16.9
9,2000,Kenny Anderson,PG,29,BOS,82,82,31.6,5.3,12.0,.440,1.0,2.7,.386,4.3,9.3,.456,.483,2.4,3.1,.775,0.7,2.1,2.7,5.1,1.7,0.1,1.6,2.8,14.0


In [10]:
rookies = df1.drop_duplicates(subset='Player', keep='first')
rookies.shape

(1944, 30)

In [11]:
rookies.columns

Index(['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [12]:
rookies2 = rookies.drop(['G', 'GS', 'FG', 'FGA', '3P','3PA', '2P', '2PA', 'FT', 'FTA', 'FT%', 'ORB','DRB', 'PF'], axis=1)
rookies2.head()

,Year,Player,Pos,Age,Tm,MP,FG%,3P%,2P%,eFG%,TRB,AST,STL,BLK,TOV,PTS
0,2000,Tariq Abdul-Wahad,SG,25,TOT,25.9,.424,.130,.435,.426,4.8,1.6,1.0,0.5,1.7,11.4
3,2000,Shareef Abdur-Rahim,SF,23,VAN,39.3,.465,.302,.478,.477,10.1,3.3,1.1,1.1,3.0,20.3
4,2000,Cory Alexander,PG,26,DEN,11.3,.286,.257,.302,.332,1.4,2.0,0.8,0.1,1.0,2.8
5,2000,Ray Allen,SG,24,MIL,37.4,.455,.423,.468,.516,4.4,3.8,1.3,0.2,2.2,22.1
6,2000,Rafer Alston,PG,23,MIL,13.4,.284,.214,.296,.300,0.9,2.6,0.4,0.0,1.1,2.2


In [13]:
rookies2['Age'] = pd.to_numeric(rookies2['Age'], errors='coerce')
rookies2['Year'] = pd.to_numeric(rookies2['Year'], errors='coerce')

In [14]:
rookies2.columns

Index(['Year', 'Player', 'Pos', 'Age', 'Tm', 'MP', 'FG%', '3P%', '2P%', 'eFG%',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS'],
      dtype='object')

In [16]:
rookies2.rename(columns={'Tm':'NBATM','3P%':'NBA3P%','2P%':'NBA2P%','MP':'NBAMIN', 'FG%':'NBAFG%', 'eFG%':'NBAeFG', 'TRB':'NBATRB', 'AST':'NBAAST', 'STL':'NBASTL', 'BLK':'NBABLK',
       'PTS':'NBAPTS','TOV':'NBATOV'},inplace=True)
rookies2.tail()

,Year,Player,Pos,Age,NBATM,NBAMIN,NBAFG%,NBA3P%,NBA2P%,NBAeFG,NBATRB,NBAAST,NBASTL,NBABLK,NBATOV,NBAPTS
11542,2019,Thomas Welsh,C,22,DEN,3.3,.538,.429,.667,.654,0.4,0.5,0.0,0.0,0.1,1.6
11550,2019,Johnathan Williams,C,23,LAL,15.5,.591,.000,.602,.591,4.1,0.5,0.3,0.3,0.7,6.5
11551,2019,Kenrich Williams,SF,24,NOP,23.5,.384,.333,.447,.477,4.8,1.8,1.0,0.4,0.8,6.1
11554,2019,Robert Williams,C,21,BOS,8.8,.706,,.706,.706,2.5,0.2,0.3,1.3,0.3,2.5
11567,2019,Trae Young,PG,20,ATL,30.9,.418,.324,.477,.480,3.7,8.1,0.9,0.2,3.8,19.1


In [17]:
maindata = []
counter=0
for name in draftees:
    counter = counter +1
    realname = name
    name = name.lower()
    name = name.replace(" ", "-")
    year = draftyear[(counter-1)]
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('tr',{'id':'players_per_game.'+str(year)})
    My_info = soup.find('div',{'id':'meta'})
    if My_table is not None:
        maindata = maindata + [realname]
        tabledata = My_table.findAll('td')
#         infodata = My_info.findAll('p')
        data = []
#         idata = []
        for cell in tabledata:
            data.append(cell.get_text())
#         for thing in infodata:
#             idata.append(thing.get_text(strip=True))
#             if idata[0][0:8]!='Position':
#                 idata.pop(0)
        maindata = maindata +data[0:26]
    print(counter, realname)

1 Kenyon Martin
2 Stromile Swift
3 Darius Miles
4 Marcus Fizer
5 Mike Miller
6 DerMarr Johnson
7 Chris Mihm
8 Jamal Crawford
9 Joel Przybilla
10 Keyon Dooling
11 Jérôme Moïso
12 Etan Thomas
13 Courtney Alexander
14 Mateen Cleaves
15 Jason Collier
16 Hedo Türkoğlu
17 Desmond Mason
18 Quentin Richardson
19 Jamaal Magloire
20 Speedy Claxton
21 Morris Peterson
22 Donnell Harvey
23 DeShawn Stevenson
24 Dalibor Bagarić
25 Jake Tsakalidis
26 Mamadou N'Diaye
27 Primož Brezec
28 Erick Barkley
29 Mark Madsen
30 Marko Jarić
31 Dan Langhi
32 A.J. Guyton
33 Jake Voskuhl
34 Khalid El-Amin
35 Mike Smith
36 Soumaila Samake
37 Eddie House
38 Eduardo Nájera
39 Lavor Postell
40 Hanno Möttölä
41 Chris Carrawell
42 Olumide Oyedeji
43 Michael Redd
44 Brian Cardinal
45 Jabari Smith
46 DeeAndre Hulett
47 Josip Sesar
48 Mark Karcher
49 Jason Hart
50 Kaniel Dickens
51 Igor Rakočević
52 Ernest Brown
53 Dan McClintock
54 Cory Hightower
55 Chris Porter
56 Jaquay Walls
57 Scoonie Penn
58 Pete Mickeal
59 Kwame Brown

466 Sammy Mejia
467 Georgios Printezis
468 D.J. Strawberry
469 Milovan Raković
470 Derrick Rose
471 Michael Beasley
472 O.J. Mayo
473 Russell Westbrook
474 Kevin Love
475 Danilo Gallinari
476 Eric Gordon
477 Joe Alexander
478 D.J. Augustin
479 Brook Lopez
480 Jerryd Bayless
481 Jason Thompson
482 Brandon Rush
483 Anthony Randolph
484 Robin Lopez
485 Marreese Speights
486 Roy Hibbert
487 JaVale McGee
488 J.J. Hickson
489 Alexis Ajinça
490 Ryan Anderson
491 Courtney Lee
492 Kosta Koufos
493 Serge Ibaka
494 Nicolas Batum
495 George Hill
496 Darrell Arthur
497 Donté Greene
498 D.J. White
499 J.R. Giddens
500 Nikola Peković
501 Walter Sharpe
502 Joey Dorsey
503 Mario Chalmers
504 DeAndre Jordan
505 Ömer Aşık
506 Luc Mbah a Moute
507 Kyle Weaver
508 Sonny Weems
509 Chris Douglas-Roberts
510 Nathan Jawai
511 Sean Singletary
512 Patrick Ewing
513 Ante Tomić
514 Goran Dragić
515 Trent Plaisted
516 Henry Walker
517 Malik Hairston
518 Richard Hendrix
519 DeVon Hardin
520 Shan Foster
521 Darnell J

918 Chris McCullough
919 Kevon Looney
920 Cedi Osman
921 Montrezl Harrell
922 Jordan Mickey
923 Anthony Brown
924 Willy Hernangómez
925 Rakeem Christmas
926 Richaun Holmes
927 Darrun Hilliard
928 Juan Pablo Vaulet
929 Josh Richardson
930 Pat Connaughton
931 Olivier Hanlan
932 Joe Young
933 Andrew Harrison
934 Marcus Thornton
935 Norman Powell
936 Artūras Gudaitis
937 Dakari Johnson
938 Aaron White
939 Marcus Eriksson
940 Tyler Harvey
941 Satnam Singh Bhamara
942 Sir'Dominic Pointer
943 Dani Díez de la Faya
944 Cady Lalanne
945 Branden Dawson
946 Nikola Radičević
947 J.P. Tokoto
948 Dimitrios Agravanis
949 Luka Mitrović
950 Ben Simmons
951 Brandon Ingram
952 Jaylen Brown
953 Dragan Bender
954 Kris Dunn
955 Buddy Hield
956 Jamal Murray
957 Marquese Chriss
958 Jakob Pöltl
959 Thon Maker
960 Domantas Sabonis
961 Taurean Waller-Prince
962 Georgios Papagiannis
963 Denzel Valentine
964 Juan Hernangómez
965 Guerschon Yabusele
966 Wade Baldwin
967 Henry Ellenson
968 Malik Beasley
969 Caris LeVe

In [18]:
otherdata = []
counter=0
for name in draftees:
    counter = counter +1
    realname = name
    name = name.lower()
    name = name.replace(" ", "-")
    year = draftyear[(counter-1)]
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_info = soup.find('div',{'id':'meta'})
    if My_info is not None:
        otherdata = otherdata + [realname]
        infodata = My_info.findAll('p')
        idata = []
        for thing in infodata:
            idata.append(thing.get_text(strip=True))
            if idata[0][0] =='(': 
                idata.pop(0)
            elif idata[0][1] =='-': 
                idata.insert(0,'N/A')
            if len(idata[0:1]) == 0:
                idata = ['N/A','N/A']
            elif len(idata[0:1]) == 1:
                idata.insert(1,'N/A')
        otherdata = otherdata +idata[0:1]
        
    print(counter, realname)

1 Kenyon Martin
2 Stromile Swift
3 Darius Miles
4 Marcus Fizer
5 Mike Miller
6 DerMarr Johnson
7 Chris Mihm
8 Jamal Crawford
9 Joel Przybilla
10 Keyon Dooling
11 Jérôme Moïso
12 Etan Thomas
13 Courtney Alexander
14 Mateen Cleaves
15 Jason Collier
16 Hedo Türkoğlu
17 Desmond Mason
18 Quentin Richardson
19 Jamaal Magloire
20 Speedy Claxton
21 Morris Peterson
22 Donnell Harvey
23 DeShawn Stevenson
24 Dalibor Bagarić
25 Jake Tsakalidis
26 Mamadou N'Diaye
27 Primož Brezec
28 Erick Barkley
29 Mark Madsen
30 Marko Jarić
31 Dan Langhi
32 A.J. Guyton
33 Jake Voskuhl
34 Khalid El-Amin
35 Mike Smith
36 Soumaila Samake
37 Eddie House
38 Eduardo Nájera
39 Lavor Postell
40 Hanno Möttölä
41 Chris Carrawell
42 Olumide Oyedeji
43 Michael Redd
44 Brian Cardinal
45 Jabari Smith
46 DeeAndre Hulett
47 Josip Sesar
48 Mark Karcher
49 Jason Hart
50 Kaniel Dickens
51 Igor Rakočević
52 Ernest Brown
53 Dan McClintock
54 Cory Hightower
55 Chris Porter
56 Jaquay Walls
57 Scoonie Penn
58 Pete Mickeal
59 Kwame Brown

466 Sammy Mejia
467 Georgios Printezis
468 D.J. Strawberry
469 Milovan Raković
470 Derrick Rose
471 Michael Beasley
472 O.J. Mayo
473 Russell Westbrook
474 Kevin Love
475 Danilo Gallinari
476 Eric Gordon
477 Joe Alexander
478 D.J. Augustin
479 Brook Lopez
480 Jerryd Bayless
481 Jason Thompson
482 Brandon Rush
483 Anthony Randolph
484 Robin Lopez
485 Marreese Speights
486 Roy Hibbert
487 JaVale McGee
488 J.J. Hickson
489 Alexis Ajinça
490 Ryan Anderson
491 Courtney Lee
492 Kosta Koufos
493 Serge Ibaka
494 Nicolas Batum
495 George Hill
496 Darrell Arthur
497 Donté Greene
498 D.J. White
499 J.R. Giddens
500 Nikola Peković
501 Walter Sharpe
502 Joey Dorsey
503 Mario Chalmers
504 DeAndre Jordan
505 Ömer Aşık
506 Luc Mbah a Moute
507 Kyle Weaver
508 Sonny Weems
509 Chris Douglas-Roberts
510 Nathan Jawai
511 Sean Singletary
512 Patrick Ewing
513 Ante Tomić
514 Goran Dragić
515 Trent Plaisted
516 Henry Walker
517 Malik Hairston
518 Richard Hendrix
519 DeVon Hardin
520 Shan Foster
521 Darnell J

918 Chris McCullough
919 Kevon Looney
920 Cedi Osman
921 Montrezl Harrell
922 Jordan Mickey
923 Anthony Brown
924 Willy Hernangómez
925 Rakeem Christmas
926 Richaun Holmes
927 Darrun Hilliard
928 Juan Pablo Vaulet
929 Josh Richardson
930 Pat Connaughton
931 Olivier Hanlan
932 Joe Young
933 Andrew Harrison
934 Marcus Thornton
935 Norman Powell
936 Artūras Gudaitis
937 Dakari Johnson
938 Aaron White
939 Marcus Eriksson
940 Tyler Harvey
941 Satnam Singh Bhamara
942 Sir'Dominic Pointer
943 Dani Díez de la Faya
944 Cady Lalanne
945 Branden Dawson
946 Nikola Radičević
947 J.P. Tokoto
948 Dimitrios Agravanis
949 Luka Mitrović
950 Ben Simmons
951 Brandon Ingram
952 Jaylen Brown
953 Dragan Bender
954 Kris Dunn
955 Buddy Hield
956 Jamal Murray
957 Marquese Chriss
958 Jakob Pöltl
959 Thon Maker
960 Domantas Sabonis
961 Taurean Waller-Prince
962 Georgios Papagiannis
963 Denzel Valentine
964 Juan Hernangómez
965 Guerschon Yabusele
966 Wade Baldwin
967 Henry Ellenson
968 Malik Beasley
969 Caris LeVe

In [19]:
otherdata[117][1]

'/'

In [20]:
otherdata
refined3 = np.reshape(otherdata, (-1, 2))

In [21]:
colhead = ['Name','College Position']
college_position = pd.DataFrame(refined3,columns=colhead)

In [22]:
college_position['College Position'] = college_position['College Position'].str.replace('Position:','')

In [23]:
college_position['College Position'].unique()

array(['N/A', 'Forward', 'Guard', 'Center', 'Guard-Forward',
       'Center-Forward'], dtype=object)

In [24]:
colheaders = ['Name','School','Conf','G','GS','MP','FG','FGA','FG%','2P','2PA','2P%','3P','3PA','3P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']

In [25]:
maindata
refined2 = np.reshape(maindata, (-1, 27))

In [26]:
college = pd.DataFrame(refined2, columns=colheaders)
college

,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Kenyon Martin,Cincinnati,CUSA,31,,29.3,7.1,12.5,.568,7.1,12.3,.573,0.1,0.2,.286,4.5,6.6,.684,,,9.7,1.4,1.4,3.5,1.8,2.3,18.9
1,Stromile Swift,LSU,SEC,34,,29.8,6.1,10.1,.608,5.9,9.3,.634,0.2,0.7,.280,3.7,6.1,.617,,,8.2,0.9,1.5,2.8,2.4,2.6,16.2
2,Marcus Fizer,Iowa State,Big 12,37,37,33.6,8.8,15.2,.582,8.4,14.1,.600,0.4,1.1,.357,4.7,6.5,.732,,,7.7,1.1,0.8,1.1,2.1,2.8,22.8
3,Mike Miller,Florida,SEC,37,,28.6,4.7,9.9,.476,3.5,6.2,.559,1.3,3.8,.338,3.4,4.6,.729,,,6.6,2.5,1.2,0.4,1.9,1.9,14.1
4,DerMarr Johnson,Cincinnati,CUSA,32,,27.5,4.4,9.2,.478,2.8,4.8,.575,1.6,4.4,.371,2.2,3.0,.737,,,3.8,1.4,1.0,0.9,1.4,2.0,12.6
5,Chris Mihm,Texas,Big 12,33,,30.7,6.2,11.9,.523,6.0,11.5,.525,0.2,0.5,.467,5.0,7.0,.707,,,10.5,0.7,0.3,2.7,2.5,2.7,17.7
6,Jamal Crawford,Michigan,Big Ten,17,,33.9,6.2,15.0,.412,4.2,9.1,.468,1.9,5.9,.327,2.4,3.0,.784,,,2.8,4.5,1.1,0.9,3.1,1.1,16.6
7,Joel Przybilla,Minnesota,Big Ten,21,,30.4,5.8,9.5,.613,5.8,9.5,.613,0.0,0.0,,2.6,5.3,.495,,,8.4,2.4,0.8,3.9,3.7,2.9,14.2
8,Keyon Dooling,Missouri,Big 12,31,,31.7,4.7,12.0,.389,2.8,6.5,.424,1.9,5.5,.347,4.0,5.4,.743,,,2.7,3.6,1.4,0.5,2.5,1.8,15.3
9,Etan Thomas,Syracuse,Big East,29,,32.4,5.1,8.5,.602,5.1,8.5,.602,0.0,0.0,,3.4,5.0,.678,,,9.3,0.6,0.8,3.7,2.0,3.2,13.6


In [62]:
collegefile = pd.read_csv('college.csv')
collegefile = collegefile[['Name', 'School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P',
       '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']]
collegefile = collegefile.drop_duplicates(subset=['Name','School'],keep='first')
print(college.shape,collegefile.shape)

(742, 27) (45, 27)


In [63]:
finalcollegedf = pd.concat([college,collegefile])
finalcollegedf 

,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Kenyon Martin,Cincinnati,CUSA,31,,29.3,7.1,12.5,.568,7.1,12.3,.573,0.1,0.2,.286,4.5,6.6,.684,,,9.7,1.4,1.4,3.5,1.8,2.3,18.9
1,Stromile Swift,LSU,SEC,34,,29.8,6.1,10.1,.608,5.9,9.3,.634,0.2,0.7,.280,3.7,6.1,.617,,,8.2,0.9,1.5,2.8,2.4,2.6,16.2
2,Marcus Fizer,Iowa State,Big 12,37,37,33.6,8.8,15.2,.582,8.4,14.1,.600,0.4,1.1,.357,4.7,6.5,.732,,,7.7,1.1,0.8,1.1,2.1,2.8,22.8
3,Mike Miller,Florida,SEC,37,,28.6,4.7,9.9,.476,3.5,6.2,.559,1.3,3.8,.338,3.4,4.6,.729,,,6.6,2.5,1.2,0.4,1.9,1.9,14.1
4,DerMarr Johnson,Cincinnati,CUSA,32,,27.5,4.4,9.2,.478,2.8,4.8,.575,1.6,4.4,.371,2.2,3.0,.737,,,3.8,1.4,1.0,0.9,1.4,2.0,12.6
5,Chris Mihm,Texas,Big 12,33,,30.7,6.2,11.9,.523,6.0,11.5,.525,0.2,0.5,.467,5.0,7.0,.707,,,10.5,0.7,0.3,2.7,2.5,2.7,17.7
6,Jamal Crawford,Michigan,Big Ten,17,,33.9,6.2,15.0,.412,4.2,9.1,.468,1.9,5.9,.327,2.4,3.0,.784,,,2.8,4.5,1.1,0.9,3.1,1.1,16.6
7,Joel Przybilla,Minnesota,Big Ten,21,,30.4,5.8,9.5,.613,5.8,9.5,.613,0.0,0.0,,2.6,5.3,.495,,,8.4,2.4,0.8,3.9,3.7,2.9,14.2
8,Keyon Dooling,Missouri,Big 12,31,,31.7,4.7,12.0,.389,2.8,6.5,.424,1.9,5.5,.347,4.0,5.4,.743,,,2.7,3.6,1.4,0.5,2.5,1.8,15.3
9,Etan Thomas,Syracuse,Big East,29,,32.4,5.1,8.5,.602,5.1,8.5,.602,0.0,0.0,,3.4,5.0,.678,,,9.3,0.6,0.8,3.7,2.0,3.2,13.6


In [64]:
merged_df = finalcollegedf.merge(rookies2, left_on='Name', right_on='Player', how='left')
merged_df

,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Player,Pos,Age,NBATM,NBAMIN,NBAFG%,NBA3P%,NBA2P%,NBAeFG,NBATRB,NBAAST,NBASTL,NBABLK,NBATOV,NBAPTS
0,Kenyon Martin,Cincinnati,CUSA,31,,29.3,7.1,12.5,.568,7.1,12.3,.573,0.1,0.2,.286,4.5,6.6,.684,,,9.7,1.4,1.4,3.5,1.8,2.3,18.9,2001.0,Kenyon Martin,PF,23.0,NJN,33.4,.445,.091,.450,.446,7.4,1.9,1.1,1.7,2.0,12.0
1,Stromile Swift,LSU,SEC,34,,29.8,6.1,10.1,.608,5.9,9.3,.634,0.2,0.7,.280,3.7,6.1,.617,,,8.2,0.9,1.5,2.8,2.4,2.6,16.2,2001.0,Stromile Swift,PF,21.0,VAN,16.4,.451,.000,.457,.451,3.6,0.4,0.8,1.0,0.8,4.9
2,Marcus Fizer,Iowa State,Big 12,37,37,33.6,8.8,15.2,.582,8.4,14.1,.600,0.4,1.1,.357,4.7,6.5,.732,,,7.7,1.1,0.8,1.1,2.1,2.8,22.8,2001.0,Marcus Fizer,PF,22.0,CHI,21.9,.430,.256,.442,.438,4.3,1.1,0.4,0.3,1.7,9.5
3,Mike Miller,Florida,SEC,37,,28.6,4.7,9.9,.476,3.5,6.2,.559,1.3,3.8,.338,3.4,4.6,.729,,,6.6,2.5,1.2,0.4,1.9,1.9,14.1,2001.0,Mike Miller,SF,20.0,ORL,29.1,.436,.407,.457,.523,4.0,1.7,0.6,0.2,1.2,11.9
4,DerMarr Johnson,Cincinnati,CUSA,32,,27.5,4.4,9.2,.478,2.8,4.8,.575,1.6,4.4,.371,2.2,3.0,.737,,,3.8,1.4,1.0,0.9,1.4,2.0,12.6,2001.0,DerMarr Johnson,SF,20.0,ATL,16.8,.374,.323,.399,.427,2.3,0.8,0.6,0.4,1.2,5.1
5,Chris Mihm,Texas,Big 12,33,,30.7,6.2,11.9,.523,6.0,11.5,.525,0.2,0.5,.467,5.0,7.0,.707,,,10.5,0.7,0.3,2.7,2.5,2.7,17.7,2001.0,Chris Mihm,C,21.0,CLE,19.8,.442,.000,.444,.442,4.7,0.3,0.3,0.9,1.4,7.6
6,Jamal Crawford,Michigan,Big Ten,17,,33.9,6.2,15.0,.412,4.2,9.1,.468,1.9,5.9,.327,2.4,3.0,.784,,,2.8,4.5,1.1,0.9,3.1,1.1,16.6,2001.0,Jamal Crawford,PG,20.0,CHI,17.2,.352,.350,.353,.419,1.5,2.3,0.7,0.2,1.4,4.6
7,Joel Przybilla,Minnesota,Big Ten,21,,30.4,5.8,9.5,.613,5.8,9.5,.613,0.0,0.0,,2.6,5.3,.495,,,8.4,2.4,0.8,3.9,3.7,2.9,14.2,2001.0,Joel Przybilla,C,21.0,MIL,8.2,.343,,.343,.343,2.2,0.1,0.1,0.9,0.4,0.8
8,Keyon Dooling,Missouri,Big 12,31,,31.7,4.7,12.0,.389,2.8,6.5,.424,1.9,5.5,.347,4.0,5.4,.743,,,2.7,3.6,1.4,0.5,2.5,1.8,15.3,2001.0,Keyon Dooling,PG,20.0,LAC,16.3,.409,.350,.426,.448,1.2,2.3,0.5,0.1,1.2,5.9
9,Etan Thomas,Syracuse,Big East,29,,32.4,5.1,8.5,.602,5.1,8.5,.602,0.0,0.0,,3.4,5.0,.678,,,9.3,0.6,0.8,3.7,2.0,3.2,13.6,2002.0,Etan Thomas,C,23.0,WAS,13.1,.536,,.536,.536,3.9,0.1,0.4,0.7,0.6,4.3


In [65]:
merged_df.columns

Index(['Name', 'School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P',
       '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'Player', 'Pos',
       'Age', 'NBATM', 'NBAMIN', 'NBAFG%', 'NBA3P%', 'NBA2P%', 'NBAeFG',
       'NBATRB', 'NBAAST', 'NBASTL', 'NBABLK', 'NBATOV', 'NBAPTS'],
      dtype='object')

In [66]:
merged_df = merged_df[['Name','Year', 'School', 'G', 'MP', 'FG', 'FGA', 'FG%', '2P',
       '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%','PTS',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',  'Pos',
        'NBATM', 'NBAPTS','NBATRB', 'NBAAST', 'NBASTL', 'NBABLK', 'NBATOV']]
merged_df

,Name,Year,School,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,PTS,TRB,AST,STL,BLK,TOV,PF,Pos,NBATM,NBAPTS,NBATRB,NBAAST,NBASTL,NBABLK,NBATOV
0,Kenyon Martin,2001.0,Cincinnati,31,29.3,7.1,12.5,.568,7.1,12.3,.573,0.1,0.2,.286,4.5,6.6,.684,18.9,9.7,1.4,1.4,3.5,1.8,2.3,PF,NJN,12.0,7.4,1.9,1.1,1.7,2.0
1,Stromile Swift,2001.0,LSU,34,29.8,6.1,10.1,.608,5.9,9.3,.634,0.2,0.7,.280,3.7,6.1,.617,16.2,8.2,0.9,1.5,2.8,2.4,2.6,PF,VAN,4.9,3.6,0.4,0.8,1.0,0.8
2,Marcus Fizer,2001.0,Iowa State,37,33.6,8.8,15.2,.582,8.4,14.1,.600,0.4,1.1,.357,4.7,6.5,.732,22.8,7.7,1.1,0.8,1.1,2.1,2.8,PF,CHI,9.5,4.3,1.1,0.4,0.3,1.7
3,Mike Miller,2001.0,Florida,37,28.6,4.7,9.9,.476,3.5,6.2,.559,1.3,3.8,.338,3.4,4.6,.729,14.1,6.6,2.5,1.2,0.4,1.9,1.9,SF,ORL,11.9,4.0,1.7,0.6,0.2,1.2
4,DerMarr Johnson,2001.0,Cincinnati,32,27.5,4.4,9.2,.478,2.8,4.8,.575,1.6,4.4,.371,2.2,3.0,.737,12.6,3.8,1.4,1.0,0.9,1.4,2.0,SF,ATL,5.1,2.3,0.8,0.6,0.4,1.2
5,Chris Mihm,2001.0,Texas,33,30.7,6.2,11.9,.523,6.0,11.5,.525,0.2,0.5,.467,5.0,7.0,.707,17.7,10.5,0.7,0.3,2.7,2.5,2.7,C,CLE,7.6,4.7,0.3,0.3,0.9,1.4
6,Jamal Crawford,2001.0,Michigan,17,33.9,6.2,15.0,.412,4.2,9.1,.468,1.9,5.9,.327,2.4,3.0,.784,16.6,2.8,4.5,1.1,0.9,3.1,1.1,PG,CHI,4.6,1.5,2.3,0.7,0.2,1.4
7,Joel Przybilla,2001.0,Minnesota,21,30.4,5.8,9.5,.613,5.8,9.5,.613,0.0,0.0,,2.6,5.3,.495,14.2,8.4,2.4,0.8,3.9,3.7,2.9,C,MIL,0.8,2.2,0.1,0.1,0.9,0.4
8,Keyon Dooling,2001.0,Missouri,31,31.7,4.7,12.0,.389,2.8,6.5,.424,1.9,5.5,.347,4.0,5.4,.743,15.3,2.7,3.6,1.4,0.5,2.5,1.8,PG,LAC,5.9,1.2,2.3,0.5,0.1,1.2
9,Etan Thomas,2002.0,Syracuse,29,32.4,5.1,8.5,.602,5.1,8.5,.602,0.0,0.0,,3.4,5.0,.678,13.6,9.3,0.6,0.8,3.7,2.0,3.2,C,WAS,4.3,3.9,0.1,0.4,0.7,0.6


In [67]:
rooks = pd.DataFrame({
    'Name':draftees,
    'Draft Position':draftposit,
    'Draft Year':draftyear})
rooks

,Name,Draft Position,Draft Year
0,Kenyon Martin,1,2000
1,Stromile Swift,2,2000
2,Darius Miles,3,2000
3,Marcus Fizer,4,2000
4,Mike Miller,5,2000
5,DerMarr Johnson,6,2000
6,Chris Mihm,7,2000
7,Jamal Crawford,8,2000
8,Joel Przybilla,9,2000
9,Keyon Dooling,10,2000


In [68]:
rook2019 = pd.read_excel('rookiedraft2019.xlsx')
rooks2019 = rook2019[['Name','Draft Position','Draft Year']]
rooksposition2019 = rook2019[['Name','Player Position']]

In [69]:
rook2019

,Name,Player Position,Draft Position,Draft Year
0,Zion Williamson,PF,1,2019
1,RJ Barrett,SG,3,2019
2,DeAndre Hunter,SF,4,2019
3,Darius Garland,PG,5,2019
4,Coby White,PG,6,2019
5,Jarrett Culver,SG,7,2019
6,Bruno Fernando,C,9,2019
7,Mfiondu Kabengele,PF,10,2019
8,Nassir Little,SF,11,2019
9,Rui Hachimura,SF,14,2019


In [70]:
rooks= pd.concat([rooks,rooks2019])
rooks

,Name,Draft Position,Draft Year
0,Kenyon Martin,1,2000
1,Stromile Swift,2,2000
2,Darius Miles,3,2000
3,Marcus Fizer,4,2000
4,Mike Miller,5,2000
5,DerMarr Johnson,6,2000
6,Chris Mihm,7,2000
7,Jamal Crawford,8,2000
8,Joel Przybilla,9,2000
9,Keyon Dooling,10,2000


In [71]:
rooksposition2019

,Name,Player Position
0,Zion Williamson,PF
1,RJ Barrett,SG
2,DeAndre Hunter,SF
3,Darius Garland,PG
4,Coby White,PG
5,Jarrett Culver,SG
6,Bruno Fernando,C
7,Mfiondu Kabengele,PF
8,Nassir Little,SF
9,Rui Hachimura,SF


In [72]:
merged_df = merged_df.merge(rooks,on='Name',how='left')

In [73]:
merged_df = merged_df.merge(rooksposition2019,on='Name',how='left')

In [74]:
merged_df['Position Type'] = merged_df['Pos'].astype(str)+merged_df['Player Position'].astype(str)

In [75]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 789 entries, 0 to 788
Data columns (total 36 columns):
Name               789 non-null object
Year               675 non-null float64
School             789 non-null object
G                  789 non-null object
MP                 789 non-null object
FG                 789 non-null object
FGA                789 non-null object
FG%                789 non-null object
2P                 789 non-null object
2PA                789 non-null object
2P%                789 non-null object
3P                 789 non-null object
3PA                789 non-null object
3P%                787 non-null object
FT                 789 non-null object
FTA                789 non-null object
FT%                789 non-null object
PTS                789 non-null object
TRB                789 non-null object
AST                789 non-null object
STL                789 non-null object
BLK                789 non-null object
TOV                789 non-null obje

In [76]:
FINAL_DF = merged_df.loc[merged_df['Draft Position'].notnull()]
FINAL_DF = FINAL_DF.drop_duplicates(subset=['Name','Year'],keep='first')
FINAL_DF

,Name,Year,School,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,PTS,TRB,AST,STL,BLK,TOV,PF,Pos,NBATM,NBAPTS,NBATRB,NBAAST,NBASTL,NBABLK,NBATOV,Draft Position,Draft Year,Player Position,Position Type
0,Kenyon Martin,2001.0,Cincinnati,31,29.3,7.1,12.5,.568,7.1,12.3,.573,0.1,0.2,.286,4.5,6.6,.684,18.9,9.7,1.4,1.4,3.5,1.8,2.3,PF,NJN,12.0,7.4,1.9,1.1,1.7,2.0,1,2000,NaN,PFnan
1,Stromile Swift,2001.0,LSU,34,29.8,6.1,10.1,.608,5.9,9.3,.634,0.2,0.7,.280,3.7,6.1,.617,16.2,8.2,0.9,1.5,2.8,2.4,2.6,PF,VAN,4.9,3.6,0.4,0.8,1.0,0.8,2,2000,NaN,PFnan
2,Marcus Fizer,2001.0,Iowa State,37,33.6,8.8,15.2,.582,8.4,14.1,.600,0.4,1.1,.357,4.7,6.5,.732,22.8,7.7,1.1,0.8,1.1,2.1,2.8,PF,CHI,9.5,4.3,1.1,0.4,0.3,1.7,4,2000,NaN,PFnan
3,Mike Miller,2001.0,Florida,37,28.6,4.7,9.9,.476,3.5,6.2,.559,1.3,3.8,.338,3.4,4.6,.729,14.1,6.6,2.5,1.2,0.4,1.9,1.9,SF,ORL,11.9,4.0,1.7,0.6,0.2,1.2,5,2000,NaN,SFnan
4,DerMarr Johnson,2001.0,Cincinnati,32,27.5,4.4,9.2,.478,2.8,4.8,.575,1.6,4.4,.371,2.2,3.0,.737,12.6,3.8,1.4,1.0,0.9,1.4,2.0,SF,ATL,5.1,2.3,0.8,0.6,0.4,1.2,6,2000,NaN,SFnan
5,Chris Mihm,2001.0,Texas,33,30.7,6.2,11.9,.523,6.0,11.5,.525,0.2,0.5,.467,5.0,7.0,.707,17.7,10.5,0.7,0.3,2.7,2.5,2.7,C,CLE,7.6,4.7,0.3,0.3,0.9,1.4,7,2000,NaN,Cnan
6,Jamal Crawford,2001.0,Michigan,17,33.9,6.2,15.0,.412,4.2,9.1,.468,1.9,5.9,.327,2.4,3.0,.784,16.6,2.8,4.5,1.1,0.9,3.1,1.1,PG,CHI,4.6,1.5,2.3,0.7,0.2,1.4,8,2000,NaN,PGnan
7,Joel Przybilla,2001.0,Minnesota,21,30.4,5.8,9.5,.613,5.8,9.5,.613,0.0,0.0,,2.6,5.3,.495,14.2,8.4,2.4,0.8,3.9,3.7,2.9,C,MIL,0.8,2.2,0.1,0.1,0.9,0.4,9,2000,NaN,Cnan
8,Keyon Dooling,2001.0,Missouri,31,31.7,4.7,12.0,.389,2.8,6.5,.424,1.9,5.5,.347,4.0,5.4,.743,15.3,2.7,3.6,1.4,0.5,2.5,1.8,PG,LAC,5.9,1.2,2.3,0.5,0.1,1.2,10,2000,NaN,PGnan
9,Etan Thomas,2002.0,Syracuse,29,32.4,5.1,8.5,.602,5.1,8.5,.602,0.0,0.0,,3.4,5.0,.678,13.6,9.3,0.6,0.8,3.7,2.0,3.2,C,WAS,4.3,3.9,0.1,0.4,0.7,0.6,12,2000,NaN,Cnan


In [77]:
FINAL_DF['Year'].fillna(value=(FINAL_DF['Draft Year']+1),inplace=True)

In [78]:
FINAL_DF.drop(columns=['Draft Year','Player Position'],inplace=True)

In [81]:
BEFORE_2019 = FINAL_DF.loc[FINAL_DF['Year']!= 2020.0]

In [82]:
AFTER_2019 = FINAL_DF.loc[FINAL_DF['Year']== 2020.0]

In [84]:
BEFORE_2019 = BEFORE_2019.loc[BEFORE_2019['NBAPTS'].notnull()]

In [87]:
FINAL_DF = pd.concat([BEFORE_2019,AFTER_2019])

In [88]:
FINAL_DF

,Name,Year,School,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,PTS,TRB,AST,STL,BLK,TOV,PF,Pos,NBATM,NBAPTS,NBATRB,NBAAST,NBASTL,NBABLK,NBATOV,Draft Position,Position Type
0,Kenyon Martin,2001.0,Cincinnati,31,29.3,7.1,12.5,.568,7.1,12.3,.573,0.1,0.2,.286,4.5,6.6,.684,18.9,9.7,1.4,1.4,3.5,1.8,2.3,PF,NJN,12.0,7.4,1.9,1.1,1.7,2.0,1,PFnan
1,Stromile Swift,2001.0,LSU,34,29.8,6.1,10.1,.608,5.9,9.3,.634,0.2,0.7,.280,3.7,6.1,.617,16.2,8.2,0.9,1.5,2.8,2.4,2.6,PF,VAN,4.9,3.6,0.4,0.8,1.0,0.8,2,PFnan
2,Marcus Fizer,2001.0,Iowa State,37,33.6,8.8,15.2,.582,8.4,14.1,.600,0.4,1.1,.357,4.7,6.5,.732,22.8,7.7,1.1,0.8,1.1,2.1,2.8,PF,CHI,9.5,4.3,1.1,0.4,0.3,1.7,4,PFnan
3,Mike Miller,2001.0,Florida,37,28.6,4.7,9.9,.476,3.5,6.2,.559,1.3,3.8,.338,3.4,4.6,.729,14.1,6.6,2.5,1.2,0.4,1.9,1.9,SF,ORL,11.9,4.0,1.7,0.6,0.2,1.2,5,SFnan
4,DerMarr Johnson,2001.0,Cincinnati,32,27.5,4.4,9.2,.478,2.8,4.8,.575,1.6,4.4,.371,2.2,3.0,.737,12.6,3.8,1.4,1.0,0.9,1.4,2.0,SF,ATL,5.1,2.3,0.8,0.6,0.4,1.2,6,SFnan
5,Chris Mihm,2001.0,Texas,33,30.7,6.2,11.9,.523,6.0,11.5,.525,0.2,0.5,.467,5.0,7.0,.707,17.7,10.5,0.7,0.3,2.7,2.5,2.7,C,CLE,7.6,4.7,0.3,0.3,0.9,1.4,7,Cnan
6,Jamal Crawford,2001.0,Michigan,17,33.9,6.2,15.0,.412,4.2,9.1,.468,1.9,5.9,.327,2.4,3.0,.784,16.6,2.8,4.5,1.1,0.9,3.1,1.1,PG,CHI,4.6,1.5,2.3,0.7,0.2,1.4,8,PGnan
7,Joel Przybilla,2001.0,Minnesota,21,30.4,5.8,9.5,.613,5.8,9.5,.613,0.0,0.0,,2.6,5.3,.495,14.2,8.4,2.4,0.8,3.9,3.7,2.9,C,MIL,0.8,2.2,0.1,0.1,0.9,0.4,9,Cnan
8,Keyon Dooling,2001.0,Missouri,31,31.7,4.7,12.0,.389,2.8,6.5,.424,1.9,5.5,.347,4.0,5.4,.743,15.3,2.7,3.6,1.4,0.5,2.5,1.8,PG,LAC,5.9,1.2,2.3,0.5,0.1,1.2,10,PGnan
9,Etan Thomas,2002.0,Syracuse,29,32.4,5.1,8.5,.602,5.1,8.5,.602,0.0,0.0,,3.4,5.0,.678,13.6,9.3,0.6,0.8,3.7,2.0,3.2,C,WAS,4.3,3.9,0.1,0.4,0.7,0.6,12,Cnan


In [89]:
FINAL_DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 718 entries, 0 to 788
Data columns (total 34 columns):
Name              718 non-null object
Year              718 non-null float64
School            718 non-null object
G                 718 non-null object
MP                718 non-null object
FG                718 non-null object
FGA               718 non-null object
FG%               718 non-null object
2P                718 non-null object
2PA               718 non-null object
2P%               718 non-null object
3P                718 non-null object
3PA               718 non-null object
3P%               716 non-null object
FT                718 non-null object
FTA               718 non-null object
FT%               718 non-null object
PTS               718 non-null object
TRB               718 non-null object
AST               718 non-null object
STL               718 non-null object
BLK               718 non-null object
TOV               718 non-null object
PF                71

In [90]:
FINAL_DF['Position Type']=FINAL_DF['Position Type'].str.replace('nan','')

In [91]:
FINAL_DF.drop(columns=['Pos'],inplace=True)

In [92]:
FINAL_DF

,Name,Year,School,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,PTS,TRB,AST,STL,BLK,TOV,PF,NBATM,NBAPTS,NBATRB,NBAAST,NBASTL,NBABLK,NBATOV,Draft Position,Position Type
0,Kenyon Martin,2001.0,Cincinnati,31,29.3,7.1,12.5,.568,7.1,12.3,.573,0.1,0.2,.286,4.5,6.6,.684,18.9,9.7,1.4,1.4,3.5,1.8,2.3,NJN,12.0,7.4,1.9,1.1,1.7,2.0,1,PF
1,Stromile Swift,2001.0,LSU,34,29.8,6.1,10.1,.608,5.9,9.3,.634,0.2,0.7,.280,3.7,6.1,.617,16.2,8.2,0.9,1.5,2.8,2.4,2.6,VAN,4.9,3.6,0.4,0.8,1.0,0.8,2,PF
2,Marcus Fizer,2001.0,Iowa State,37,33.6,8.8,15.2,.582,8.4,14.1,.600,0.4,1.1,.357,4.7,6.5,.732,22.8,7.7,1.1,0.8,1.1,2.1,2.8,CHI,9.5,4.3,1.1,0.4,0.3,1.7,4,PF
3,Mike Miller,2001.0,Florida,37,28.6,4.7,9.9,.476,3.5,6.2,.559,1.3,3.8,.338,3.4,4.6,.729,14.1,6.6,2.5,1.2,0.4,1.9,1.9,ORL,11.9,4.0,1.7,0.6,0.2,1.2,5,SF
4,DerMarr Johnson,2001.0,Cincinnati,32,27.5,4.4,9.2,.478,2.8,4.8,.575,1.6,4.4,.371,2.2,3.0,.737,12.6,3.8,1.4,1.0,0.9,1.4,2.0,ATL,5.1,2.3,0.8,0.6,0.4,1.2,6,SF
5,Chris Mihm,2001.0,Texas,33,30.7,6.2,11.9,.523,6.0,11.5,.525,0.2,0.5,.467,5.0,7.0,.707,17.7,10.5,0.7,0.3,2.7,2.5,2.7,CLE,7.6,4.7,0.3,0.3,0.9,1.4,7,C
6,Jamal Crawford,2001.0,Michigan,17,33.9,6.2,15.0,.412,4.2,9.1,.468,1.9,5.9,.327,2.4,3.0,.784,16.6,2.8,4.5,1.1,0.9,3.1,1.1,CHI,4.6,1.5,2.3,0.7,0.2,1.4,8,PG
7,Joel Przybilla,2001.0,Minnesota,21,30.4,5.8,9.5,.613,5.8,9.5,.613,0.0,0.0,,2.6,5.3,.495,14.2,8.4,2.4,0.8,3.9,3.7,2.9,MIL,0.8,2.2,0.1,0.1,0.9,0.4,9,C
8,Keyon Dooling,2001.0,Missouri,31,31.7,4.7,12.0,.389,2.8,6.5,.424,1.9,5.5,.347,4.0,5.4,.743,15.3,2.7,3.6,1.4,0.5,2.5,1.8,LAC,5.9,1.2,2.3,0.5,0.1,1.2,10,PG
9,Etan Thomas,2002.0,Syracuse,29,32.4,5.1,8.5,.602,5.1,8.5,.602,0.0,0.0,,3.4,5.0,.678,13.6,9.3,0.6,0.8,3.7,2.0,3.2,WAS,4.3,3.9,0.1,0.4,0.7,0.6,12,C


In [93]:
FINAL_DF.to_csv('Final_Combined_DF.csv')
